# TensorFlow

## 1 - Exploring the Tensorflow Library

In [3]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

%matplotlib inline
np.random.seed(1)

$$loss = \mathcal{L}(\hat{y}, y) = (\hat y^{(i)} - y^{(i)})^2 \tag{1}$$

In [4]:
a = tf.constant(36)
b = tf.constant(39)
loss = tf.Variable(tf.zeros(shape=(1)), name='loss')  # Create a variable for the loss

def compute_loss(y, y_hat):
    return (y - y_hat)**2

print(compute_loss(a, b))
tf.print(compute_loss(a, b))

tf.Tensor(9, shape=(), dtype=int32)
9


In [5]:
a = tf.constant(2)
b = tf.constant(10)
print(tf.multiply(a,b))
tf.print(tf.multiply(a,b))

tf.Tensor(20, shape=(), dtype=int32)
20


### 1.1 - Linear function

Compute $WX + b$

In [6]:
def linear_function():
    """
    Implements a linear function: 
            Initializes X to be a random tensor of shape (3,1)
            Initializes W to be a random tensor of shape (4,3)
            Initializes b to be a random tensor of shape (4,1)
    Returns: 
    result -- runs the session for Y = WX + b 
    """
    
    np.random.seed(1)
    
    """
    Note, to ensure that the "random" numbers generated match the expected results,
    please create the variables in the order given in the starting code below.
    (Do not re-arrange the order).
    """
    X = tf.constant(np.random.randn(3,1), name = "X")
    W = tf.constant(np.random.randn(4,3), name = "W")
    b = tf.constant(np.random.randn(4,1), name = "b")
    Y = tf.Variable(tf.add(tf.matmul(W, X), b), name='Y')
    
    return Y

In [7]:
tf.print(linear_function())

[[-2.1565738152972127]
 [2.9589144574519342]
 [-1.0892678106543534]
 [-0.84538042215143838]]


### 1.2 - Computing the sigmoid 

In [32]:
def sigmoid(z):
    """
    Computes the sigmoid of z
    
    Arguments:
    z -- input value, scalar or vector
    
    Returns: 
    results -- the sigmoid of z
    """

    z = tf.cast(z, tf.float32)
    return tf.sigmoid(z, name=None)

In [33]:
tf.print(sigmoid(12))

0.999993801


### 1.3 -  Computing the Cost

$$ J = - \frac{1}{m}  \sum_{i = 1}^m  \large ( \small y^{(i)} \log a^{ [2] (i)} + (1-y^{(i)})\log (1-a^{ [2] (i)} )\large )\small\tag{2}$$

In [93]:
def cost(logits, labels):
    """
    Computes the cost using the sigmoid cross entropy
    
    Arguments:
    logits -- vector containing z, output of the last linear unit (before the final sigmoid activation)
    labels -- vector of labels y (1 or 0) 
    
    Note: What we've been calling "z" and "y" in this class are respectively called "logits" and "labels" 
    in the TensorFlow documentation. So logits will feed into z, and labels into y. 
    
    Returns:
    cost -- runs the session of the cost (formula (2))
    """
    
    # Create the placeholders for "logits" (z) and "labels" (y) (approx. 2 lines)
    z = tf.placeholder(tf.float32, name = "z")
    y = tf.placeholder(tf.float32, name = "y")
    
    # Use the loss function (approx. 1 line)
    cost = tf.nn.sigmoid_cross_entropy_with_logits(logits = z,  labels = y)
    
    # Create a session (approx. 1 line). See method 1 above.
    #sess = tf.Session()
    
    # Run the session (approx. 1 line).
    #cost = sess.run(cost, feed_dict = {logits: z, labels: y})
    
    # Close the session (approx. 1 line). See method 1 above.
    #sess.close()
    
    init = tf.global_variables_initializer()         
    with tf.Session() as session:                    
        session.run(init)                            
        cost = session.run(cost, feed_dict = {z: logits, y:labels})
        session.close()
    
    return cost

In [94]:
logits = np.array([0.2,0.4,0.7,0.9])

cost = cost(logits, np.array([0,0,1,1]))
print ("cost = " + str(cost))

AttributeError: module 'tensorflow' has no attribute 'placeholder'

### 1.4 - Using One Hot encodings

<img src="tensorflow-images/onehot.png" style="width:600px;height:150px;">

In [95]:
def one_hot_matrix(labels, C):
    """
    Creates a matrix where the i-th row corresponds to the ith class number and the jth column
                     corresponds to the jth training example. So if example j had a label i. Then entry (i,j) 
                     will be 1. 
                     
    Arguments:
    labels -- vector containing the labels 
    C -- number of classes, the depth of the one hot dimension
    
    Returns: 
    one_hot -- one hot matrix
    """
    
    # Create a tf.constant equal to C (depth), name it 'C'. (approx. 1 line)
    C = tf.constant(C, name = "C")
    
    # Use tf.one_hot, be careful with the axis (approx. 1 line)
    one_hot_matrix = tf.one_hot(labels, C, 1)
    
    # Create the session (approx. 1 line)
    #sess = None
    
    # Run the session (approx. 1 line)
    #one_hot = None
    
    # Close the session (approx. 1 line). See method 1 above.
    #None
    
    init = tf.global_variables_initializer()         
    with tf.Session() as session:                    
        session.run(init)          
        one_hot_matrix = tf.transpose(one_hot_matrix)
        one_hot = session.run(one_hot_matrix)
        session.close()
    
    return one_hot

In [96]:
labels = np.array([1,2,3,0,2,1])
one_hot = one_hot_matrix(labels, C = 4)
print ("one_hot = \n" + str(one_hot))

AttributeError: module 'tensorflow' has no attribute 'global_variables_initializer'

### 1.5 - Initialize with zeros and ones

In [97]:
def ones(shape):
    """
    Creates an array of ones of dimension shape
    
    Arguments:
    shape -- shape of the array you want to create
        
    Returns: 
    ones -- array containing only ones
    """
    
    # Create "ones" tensor using tf.ones(...). (approx. 1 line)
    ones = tf.ones(shape)
    
    # Create the session (approx. 1 line)
    sess = tf.Session()
    
    # Run the session to compute 'ones' (approx. 1 line)
    ones = sess.run(ones)
    
    # Close the session (approx. 1 line). See method 1 above.
    sess.close()
    
    return ones

In [98]:
print ("ones = " + str(ones([3])))

AttributeError: module 'tensorflow' has no attribute 'Session'